In [2]:
from langgraph.graph import StateGraph,START,END
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from typing import TypedDict, Annotated
from langgraph.checkpoint.memory import MemorySaver
import os
import operator
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage
load_dotenv()

True

In [3]:
groq_api_key=os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")

In [4]:
class bot_State(TypedDict):

    messages : Annotated[list[BaseMessage],add_messages]

In [5]:
def chat_node(state : bot_State):
    messages = state['messages']

    reponse = llm.invoke(messages)

    return {"messages":[reponse]}

In [6]:
graph = StateGraph(bot_State)

checkpointer = MemorySaver()
graph.add_node("chat_node",chat_node)
graph.add_edge(START,"chat_node")
graph.add_edge("chat_node",END)

workflow = graph.compile(checkpointer=checkpointer)

In [7]:
thread_id = '1'
config = {'configurable':{'thread_id':thread_id}}

while True:
    user_input = input("type here :")
    print("user message :",user_input)

    if user_input.strip().lower() in ['exit','bye','quit']:
        break

    response = workflow.invoke({'messages':[HumanMessage(content=user_input)]},config=config)

    print ("AI :", response)


user message : hi
AI : {'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='03d8b4ac-1287-42a9-8227-baec2052022a'), AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 36, 'total_tokens': 44, 'completion_time': 0.006119242, 'completion_tokens_details': None, 'prompt_time': 0.001585532, 'prompt_tokens_details': None, 'queue_time': 0.055414928, 'total_time': 0.007704774}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3315-2499-7f60-8da2-94e0b21a1149-0', usage_metadata={'input_tokens': 36, 'output_tokens': 8, 'total_tokens': 44})]}
user message : exit
